In [29]:
%matplotlib inline
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.models import model_from_json
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.optimizers import SGD

from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score

from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

import os
import keras
print keras.__version__

import cv2

2.0.2


In [30]:
# Doing some configuration
img_width, img_height = 100,100
# Dir for train and test
train_data_dir = 'data2/64train'
validation_data_dir = 'data2/64test'
nb_train_samples = 438
nb_validation_samples = 196
epochs = 1
running_times = 1
batch_size = (nb_train_samples // epochs) * running_times
print batch_size
input_shape = None
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)    

438


In [32]:
generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=K.image_data_format())

train_generator = generator.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_train_samples,
    class_mode='binary')

validation_generator = generator.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_validation_samples,
    class_mode='binary')

Found 438 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [36]:
class MyClassifier:
    def __init__(self):
        self.vgg16 = keras.applications.vgg16.VGG16(include_top=False,
                                                    weights='imagenet',
                                                    input_shape=input_shape)
        self.tree = XGBClassifier(max_depth=20, 
                                  learning_rate=0.1,                                           
                                  n_estimators=2000,
                                  nthread=8)
        
        self.svc = SVC(probability=True)
        self.sgd = SGDClassifier(loss='log')
        self.ensemble = VotingClassifier(estimators=[
            ('svm', self.svc),
            ('sgd', self.sgd),
            ('xgboost', self.tree)
        ], voting='soft')
    
    def __preproc__(self, X):
#         X = X / 255.
        X = self.vgg16.predict(X)
        X = X.reshape([len(X), -1])
        return X
        
    def fit(self, X, y):
        X = self.__preproc__(X)
        if X is not None:
            self.tree.fit(X, y)
        
    def predict(self, X):
        X = self.__preproc__(X)
        return self.tree.predict(X)
    
    def predict_proba(self, X):
        X = self.__preproc__(X)
        return self.tree.predict_proba(X)
    
clf = MyClassifier()

# Treino

In [37]:
X_test, y_test = validation_generator.next()

In [38]:
X, y = train_generator.next()
clf.fit(X, y)
accuracy = accuracy_score(y_test, clf.predict(X_test))
print 'accuracy: %2f' % (accuracy)

accuracy: 0.806122


In [43]:
def load_image(path_img):
    img = image.load_img(path_img, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return x

def load_image_from_cv(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return x

def make_heatmap(model, imgPath, chopsize, stridesize, option):
    img = cv2.imread(imgPath)
    imgHeight, imgWidth, rgb = img.shape
    img_overlay = np.zeros((imgHeight,imgWidth,3), np.uint8)
    xposition = 0
    yposition = 0    
    while(xposition < imgWidth):
        while(yposition < imgHeight):
            if((yposition + chopsize) < imgHeight and (xposition + chopsize) < imgWidth):
                imgcrop = img[yposition : yposition+chopsize, xposition: xposition + chopsize]
                img_resize = cv2.resize(imgcrop, (img_width, img_height)) 
                img_preds = load_image_from_cv(img_resize)
                preds = model.predict_proba(img_preds)
                softmax = preds[0][1]
                print(preds,softmax)
                if(option == "weighted"):
                    weight = int(round(softmax * 255))
                    h = len(img_overlay[yposition : yposition+chopsize, xposition: xposition + chopsize])
                    w = len(img_overlay[yposition : yposition+chopsize, xposition: xposition + chopsize][0])
                    for y in range(h):
                        for x in range(w):
                            img_overlay[yposition + y, xposition + x] = img_overlay[yposition + y, xposition + x] + (weight, weight, weight)
                if(option == "normal"):
                    if(softmax > 0.5):
                        h = len(img_overlay[yposition : yposition+chopsize, xposition: xposition + chopsize])
                        w = len(img_overlay[yposition : yposition+chopsize, xposition: xposition + chopsize][0])
                        for y in range(h):
                            for x in range(w):
                                img_overlay[yposition + y, xposition + x] = img_overlay[yposition + y, xposition + x] + (255, 255, 255)
#                 predmatrix[y][x] = preds
            yposition = yposition + stridesize
        xposition = xposition + stridesize
        yposition = 0
    img_heatmap = cv2.addWeighted(img,0.5,img_overlay,0.5,0)
    file_name = imgPath[:-4]
    file_extension = imgPath[-4:]
    cv2.imwrite(file_name + "_heatmap" + file_extension, img_heatmap)
    print ("Heatmap generated on ",file_name + "_heatmap" + file_extension)

In [49]:
chopsize = 200
stridesize = 100
#Choose any image to run the heatmap
# path = "waldo_test2.jpg"
path = 'test2.jpg'

make_heatmap(clf, 'test1.jpg', chopsize, stridesize, "weighted")
print 1
# make_heatmap(clf, 'test2.jpg', chopsize, stridesize, "normal")
# print 2
# make_heatmap(clf, 'test3.jpg', chopsize, stridesize, "normal")
# print 3
# make_heatmap(clf, 'test4.jpg', chopsize, stridesize, "normal")
# print 4
# make_heatmap(clf, 'test5.jpg', chopsize, stridesize, "normal")
# print 5


(array([[ 0.99568588,  0.0043141 ]], dtype=float32), 0.004314098)
(array([[ 0.9809655 ,  0.01903451]], dtype=float32), 0.019034507)
(array([[ 0.8992517 ,  0.10074828]], dtype=float32), 0.10074828)
(array([[ 0.10710847,  0.89289153]], dtype=float32), 0.89289153)
(array([[ 0.52763337,  0.47236663]], dtype=float32), 0.47236663)
(array([[ 0.96898234,  0.03101766]], dtype=float32), 0.031017657)
(array([[ 0.02790976,  0.97209024]], dtype=float32), 0.97209024)
(array([[ 0.23248601,  0.76751399]], dtype=float32), 0.76751399)
(array([[ 0.74765319,  0.25234681]], dtype=float32), 0.25234681)
(array([[ 0.92653066,  0.07346933]], dtype=float32), 0.073469333)
(array([[ 0.98628074,  0.01371928]], dtype=float32), 0.01371928)
(array([[ 0.99674338,  0.00325664]], dtype=float32), 0.0032566446)
(array([[ 0.99661219,  0.0033878 ]], dtype=float32), 0.0033878023)
(array([[ 0.99203515,  0.00796485]], dtype=float32), 0.0079648504)
(array([[ 0.61786449,  0.38213551]], dtype=float32), 0.38213551)
(array([[ 0.986

(array([[ 0.97722811,  0.0227719 ]], dtype=float32), 0.022771897)
(array([[ 0.99554032,  0.00445969]], dtype=float32), 0.004459694)
(array([[ 0.91957951,  0.08042049]], dtype=float32), 0.080420494)
(array([[ 0.97476763,  0.02523238]], dtype=float32), 0.025232378)
(array([[ 0.98120266,  0.01879735]], dtype=float32), 0.018797345)
(array([[ 0.91742051,  0.08257952]], dtype=float32), 0.082579516)
(array([[ 0.99528098,  0.004719  ]], dtype=float32), 0.004719005)
(array([[ 0.99197716,  0.00802282]], dtype=float32), 0.0080228215)
(array([[ 0.99697208,  0.0030279 ]], dtype=float32), 0.0030279027)
(array([[ 0.97857273,  0.02142726]], dtype=float32), 0.021427255)
(array([[ 0.98342782,  0.01657217]], dtype=float32), 0.016572166)
(array([[ 0.99209493,  0.00790508]], dtype=float32), 0.0079050772)
(array([[ 0.78892326,  0.21107675]], dtype=float32), 0.21107675)
(array([[ 0.9750936 ,  0.02490641]], dtype=float32), 0.024906406)
(array([[ 0.98589933,  0.01410065]], dtype=float32), 0.014100648)
(array([

(array([[ 0.99266475,  0.00733525]], dtype=float32), 0.0073352535)
(array([[ 0.99677169,  0.00322831]], dtype=float32), 0.0032283082)
(array([[ 0.9963116 ,  0.00368838]], dtype=float32), 0.0036883806)
(array([[ 0.98492283,  0.0150772 ]], dtype=float32), 0.015077195)
(array([[ 0.99784911,  0.00215092]], dtype=float32), 0.0021509163)
(array([[ 0.98933285,  0.01066716]], dtype=float32), 0.01066716)
(array([[ 0.97754145,  0.02245857]], dtype=float32), 0.022458566)
(array([[ 0.9973982 ,  0.00260179]], dtype=float32), 0.0026017879)
(array([[ 0.98386896,  0.01613102]], dtype=float32), 0.016131025)
(array([[ 0.99825281,  0.0017472 ]], dtype=float32), 0.0017471953)
(array([[ 0.97758693,  0.02241306]], dtype=float32), 0.022413056)
(array([[ 0.99427301,  0.00572702]], dtype=float32), 0.0057270196)
(array([[ 0.99376971,  0.0062303 ]], dtype=float32), 0.0062303022)
(array([[ 0.69652963,  0.30347034]], dtype=float32), 0.30347034)
(array([[ 0.99648535,  0.00351463]], dtype=float32), 0.0035146263)
(ar

(array([[ 0.3575443,  0.6424557]], dtype=float32), 0.6424557)
(array([[ 0.98981905,  0.01018098]], dtype=float32), 0.010180977)
(array([[ 0.07020301,  0.92979699]], dtype=float32), 0.92979699)
(array([[ 0.9392913 ,  0.06070869]], dtype=float32), 0.060708694)
(array([[ 0.99507928,  0.00492074]], dtype=float32), 0.0049207429)
(array([[ 0.94701648,  0.05298355]], dtype=float32), 0.052983552)
(array([[ 0.97077787,  0.02922215]], dtype=float32), 0.029222148)
(array([[ 0.99520189,  0.00479812]], dtype=float32), 0.0047981241)
(array([[ 0.98099273,  0.01900728]], dtype=float32), 0.019007284)
(array([[ 0.98763692,  0.01236307]], dtype=float32), 0.01236307)
(array([[ 0.93981916,  0.06018084]], dtype=float32), 0.060180839)
(array([[ 0.99656612,  0.00343388]], dtype=float32), 0.0034338764)
(array([[ 0.99849677,  0.00150322]], dtype=float32), 0.0015032206)
(array([[ 0.96430945,  0.03569056]], dtype=float32), 0.035690561)
(array([[ 0.99303758,  0.0069624 ]], dtype=float32), 0.0069624009)
(array([[ 0

(array([[ 0.4852351,  0.5147649]], dtype=float32), 0.5147649)
(array([[ 0.9158836 ,  0.08411642]], dtype=float32), 0.084116422)
(array([[ 0.97689033,  0.02310969]], dtype=float32), 0.023109691)
(array([[  9.99114335e-01,   8.85657617e-04]], dtype=float32), 0.00088565762)
(array([[ 0.99486428,  0.00513573]], dtype=float32), 0.0051357332)
(array([[ 0.15967983,  0.84032017]], dtype=float32), 0.84032017)
(array([[ 0.79075992,  0.20924006]], dtype=float32), 0.20924006)
(array([[ 0.65896869,  0.34103134]], dtype=float32), 0.34103134)
(array([[ 0.16154695,  0.83845305]], dtype=float32), 0.83845305)
(array([[ 0.99272388,  0.00727615]], dtype=float32), 0.0072761467)
(array([[ 0.96172541,  0.03827456]], dtype=float32), 0.038274564)
(array([[ 0.97687846,  0.02312154]], dtype=float32), 0.023121536)
(array([[ 0.22416365,  0.77583635]], dtype=float32), 0.77583635)
(array([[ 0.90062094,  0.09937908]], dtype=float32), 0.099379078)
(array([[ 0.27381343,  0.72618657]], dtype=float32), 0.72618657)
(array

...

(array([[ 0.99185783,  0.00814216]], dtype=float32), 0.0081421649)
(array([[ 0.72285134,  0.27714866]], dtype=float32), 0.27714866)
(array([[ 0.43881971,  0.56118029]], dtype=float32), 0.56118029)
(array([[ 0.21207553,  0.78792447]], dtype=float32), 0.78792447)
(array([[ 0.9927839 ,  0.00721611]], dtype=float32), 0.0072161118)
(array([[ 0.25848913,  0.74151087]], dtype=float32), 0.74151087)
(array([[ 0.99027127,  0.00972875]], dtype=float32), 0.0097287465)
(array([[ 0.99235171,  0.0076483 ]], dtype=float32), 0.0076482953)
(array([[ 0.97915971,  0.02084032]], dtype=float32), 0.020840315)
(array([[ 0.35963917,  0.64036083]], dtype=float32), 0.64036083)
(array([[ 0.2377708,  0.7622292]], dtype=float32), 0.7622292)
(array([[ 0.00981694,  0.99018306]], dtype=float32), 0.99018306)
(array([[ 0.71749836,  0.28250164]], dtype=float32), 0.28250164)
(array([[ 0.99187613,  0.00812389]], dtype=float32), 0.0081238858)
(array([[ 0.60312402,  0.39687601]], dtype=float32), 0.39687601)
(array([[ 0.65278

(array([[ 0.58684444,  0.41315559]], dtype=float32), 0.41315559)
(array([[ 0.62516481,  0.37483519]], dtype=float32), 0.37483519)
(array([[ 0.32760578,  0.67239422]], dtype=float32), 0.67239422)
(array([[ 0.04976428,  0.95023572]], dtype=float32), 0.95023572)
(array([[ 0.18955505,  0.81044495]], dtype=float32), 0.81044495)
(array([[ 0.52786553,  0.4721345 ]], dtype=float32), 0.4721345)
(array([[ 0.53675866,  0.46324137]], dtype=float32), 0.46324137)
(array([[ 0.0068121,  0.9931879]], dtype=float32), 0.9931879)
(array([[ 0.18650424,  0.81349576]], dtype=float32), 0.81349576)
(array([[ 0.99785239,  0.00214763]], dtype=float32), 0.0021476296)
(array([[ 0.93928635,  0.06071363]], dtype=float32), 0.06071363)
(array([[ 0.68700504,  0.31299493]], dtype=float32), 0.31299493)
(array([[ 0.97092861,  0.02907138]], dtype=float32), 0.029071381)
(array([[ 0.37120378,  0.62879622]], dtype=float32), 0.62879622)
(array([[ 0.39951032,  0.60048968]], dtype=float32), 0.60048968)
(array([[ 0.438761,  0.561

(array([[ 0.08135539,  0.91864461]], dtype=float32), 0.91864461)
(array([[ 0.28975391,  0.71024609]], dtype=float32), 0.71024609)
(array([[ 0.93449402,  0.06550599]], dtype=float32), 0.065505989)
(array([[ 0.05998933,  0.94001067]], dtype=float32), 0.94001067)
(array([[ 0.64592934,  0.35407066]], dtype=float32), 0.35407066)
(array([[ 0.2156738,  0.7843262]], dtype=float32), 0.7843262)
(array([[ 0.86034322,  0.1396568 ]], dtype=float32), 0.1396568)
(array([[ 0.94704401,  0.05295601]], dtype=float32), 0.052956007)
(array([[ 0.79818904,  0.20181099]], dtype=float32), 0.20181099)
(array([[ 0.73261368,  0.26738629]], dtype=float32), 0.26738629)
(array([[ 0.9917379 ,  0.00826209]], dtype=float32), 0.0082620867)
(array([[ 0.99506193,  0.00493806]], dtype=float32), 0.0049380623)
(array([[ 0.70237815,  0.29762188]], dtype=float32), 0.29762188)
(array([[ 0.83736014,  0.16263983]], dtype=float32), 0.16263983)
(array([[ 0.59458268,  0.40541732]], dtype=float32), 0.40541732)
(array([[ 0.63090086,  

(array([[ 0.81732792,  0.1826721 ]], dtype=float32), 0.1826721)
(array([[ 0.57893467,  0.4210653 ]], dtype=float32), 0.4210653)
(array([[ 0.94697845,  0.05302158]], dtype=float32), 0.053021576)
(array([[ 0.58007455,  0.41992548]], dtype=float32), 0.41992548)
(array([[ 0.21467584,  0.78532416]], dtype=float32), 0.78532416)
(array([[ 0.42988217,  0.57011783]], dtype=float32), 0.57011783)
(array([[ 0.54299903,  0.457001  ]], dtype=float32), 0.457001)
(array([[ 0.41159838,  0.58840162]], dtype=float32), 0.58840162)
(array([[ 0.73333836,  0.26666167]], dtype=float32), 0.26666167)
(array([[ 0.40578657,  0.59421343]], dtype=float32), 0.59421343)
(array([[ 0.29690504,  0.70309496]], dtype=float32), 0.70309496)
(array([[ 0.25952584,  0.74047416]], dtype=float32), 0.74047416)
(array([[ 0.96799374,  0.03200627]], dtype=float32), 0.032006267)
(array([[ 0.99769193,  0.00230806]], dtype=float32), 0.0023080606)
(array([[ 0.88079709,  0.11920292]], dtype=float32), 0.11920292)
(array([[ 0.908328,  0.09

(array([[ 0.90439492,  0.09560509]], dtype=float32), 0.09560509)
(array([[ 0.99674636,  0.00325364]], dtype=float32), 0.0032536448)
(array([[ 0.96018428,  0.0398157 ]], dtype=float32), 0.039815702)
(array([[ 0.87536412,  0.12463587]], dtype=float32), 0.12463587)
(array([[ 0.20833552,  0.79166448]], dtype=float32), 0.79166448)
(array([[ 0.99617457,  0.00382542]], dtype=float32), 0.0038254212)
(array([[ 0.95793754,  0.04206247]], dtype=float32), 0.042062465)
(array([[ 0.95767266,  0.04232733]], dtype=float32), 0.042327333)
(array([[ 0.48558164,  0.51441836]], dtype=float32), 0.51441836)
(array([[ 0.80177796,  0.19822207]], dtype=float32), 0.19822207)
(array([[ 0.71388769,  0.28611234]], dtype=float32), 0.28611234)
(array([[ 0.68404055,  0.31595948]], dtype=float32), 0.31595948)
(array([[ 0.88598478,  0.11401524]], dtype=float32), 0.11401524)
(array([[ 0.94318384,  0.05681615]], dtype=float32), 0.05681615)
(array([[ 0.99385631,  0.00614371]], dtype=float32), 0.0061437087)
(array([[ 0.8474